**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
sudo docker run --rm -v <ruta a mi directorio>:/datos --cap-add SYS_ADMIN --privileged --name jupyterlab-numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

**Para ejecutar el comando de linux [perf](https://github.com/torvalds/linux/tree/master/tools/perf) obsérvese que es necesario correr el contenedor de docker descrito antes con las *flags* `--cap-add SYS_ADMIN` y `--privileged`.**

---

Nota generada a partir de [liga](https://www.dropbox.com/s/fyqwiqasqaa3wlt/3.1.1.Multiplicacion_de_matrices_y_estructura_de_datos.pdf?dl=0), [liga2](https://www.dropbox.com/s/l4hq45rj860ql9f/3.1.2.Localidad_y_vectorizacion.Analisis_del_error_en_computos_matriciales_basicos.pdf?dl=0)

# 3.1 El cómputo matricial y el álgebra lineal. Vectorización, BLAS y el uso del caché eficientemente.

El cómputo matricial está construído sobre una jerarquía de operaciones del álgebra lineal:

* Productos punto involucran operaciones escalares de suma y multiplicación (nivel BLAS 1).

* La multiplicación matriz-vector está hecha de productos punto (nivel BLAS 2).

* La multiplicación matriz-matriz utiliza colecciones de productos matriz-vector (nivel BLAS 3).

Las operaciones anteriores se describen en el álgebra lineal con la teoría de espacios vectoriales pero también es posible describirlas en una forma algorítmica. Ambas descripciones se complementan una a la otra.

Manejaremos nombres que en el [Linear Algebra Package: LAPACK](http://www.netlib.org/lapack/explore-html/dir_fa94b7b114d387a7a8beb2e3e22bf78d.html) son utilizados para denotar algunas operaciones con escalares, vectores o matrices. Ver [ Reference-LAPACK / lapack](https://github.com/Reference-LAPACK/lapack) para su github.

## Operación del producto interno estándar o producto punto

Consideramos $x,y \in \mathbb{R}^n$. El producto punto entre $x$ y $y$ es $c = x^Ty = \displaystyle \sum_{i=1}^n x_iy_i$. 

**Ejemplo y algoritmo del producto punto:**

In [2]:
c=0
n=5
x=[-1]*n
y=[1.5]*n

for i in range(n):
    c += x[i]*y[i]

In [3]:
c

-7.5

**Obs:**

* El producto punto de dos $n$-vectores involucran $n$ multiplicaciones y $n$ sumas para un total de $2n$ operaciones. Usamos la notación $\mathcal{O}(\cdot)$ para escribir que el producto punto es $\mathcal{O}(n)$ y se lee "de orden $n$ o proporcional a $n$" para indicar que la **cantidad de trabajo** tiene un comportamiento **lineal** con la dimensión $n$. También tal cantidad de trabajo opera sobre una **cantidad lineal de datos**.

## Operación **saxpy**

Consideramos $\alpha \in \mathbb{R}, x,y \in \mathbb{R}^n$. El nombre lo recibe por *scalar alpha x plus y*. En LAPACK se escribe en forma *update*:

$$y=\alpha x + y \therefore y_i = \alpha x_i + y_i \forall i=1,...,n$$

**Obs:** 

* El símbolo $=$ no se utiliza como igualdad de expresiones sino como en computación para denotar asignación.

* También encontramos en LAPACK `caxpy` o `daxpy` para el caso complejo y para números en doble precisión respectivamente.

* Ésta operación realiza un trabajo de $\mathcal{O}(n)$ sobre una cantidad de datos $\mathcal{O}(n)$.

**Ejemplo y algoritmo de saxpy:**

In [4]:
alpha=2
n=5
x=[-2]*n
y=[0]*n

for i in range(n):
    y[i] += alpha*x[i]

In [5]:
y

[-4, -4, -4, -4, -4]

o en una forma *update*:

In [6]:
alpha=2
n=5
x=[-2]*n
y=[3,4,-1,0,1]

for i in range(n):
    y[i] += alpha*x[i]

In [7]:
y

[-1, 0, -5, -4, -3]

**Comentario:** La operación de producto punto y *saxpy* son algoritmos catalogados como de **nivel BLAS 1**, ver [BLAS: Basic Linear Algebra Subprograms](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms). Éstos algoritmos se caracterizan por involucrar una cantidad de trabajo lineal sobre una cantidad lineal de datos. Ver [level 1](http://www.netlib.org/blas/#_level_1) para más ejemplos.

## Multiplicación matriz-vector

Consideramos $A \in \mathbb{R}^{m \times n}, x \in \mathbb{R}^n, y \in \mathbb{R}^m$. La operación $y = y + Ax$ es una operación *generalizada* saxpy, por ello se denomina **gaxpy** pero en LAPACK podemos encontrar esta operación con nombres como [sgemv](http://www.netlib.org/lapack/explore-html/db/d58/sgemv_8f.html), [dgemv](http://www.netlib.org/lapack/explore-html/dc/da8/dgemv_8f.html), [cgemv](http://www.netlib.org/lapack/explore-html/d4/d8a/cgemv_8f.html) o [zgemv](http://www.netlib.org/lapack/explore-html/db/d40/zgemv_8f.html) para los casos de precisión simple, doble o números complejos respectivamente. Hay diferentes formas de visualizar y escribir el algoritmo de multiplicación matriz-vector. Por ejemplo para una matriz $A$ con entradas:


In [8]:
m=2
n=5
A=[[1.2]*n if i%2==0 else [1]*n for i in range(m)]

In [9]:
A

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]

In [10]:
A[0][0]

1.2

In [11]:
A[1][n-1]

1

se tiene:

### Algoritmo gaxpy *row oriented*

In [12]:
x=[2]*n
y=[0]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [13]:
y

[12.0, 10]

Si $y$ tiene valores distintos de $0$, se realiza un *update*:

In [14]:
x=[2]*n
y=[-1]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [15]:
y

[11.0, 9]

**Comentarios:**

* En la versión *row oriented* del algoritmo *gaxpy*, el **inner loop** realiza **productos punto** entre el $i$-ésimo renglón de $A$ y el vector $x$. Se realizan $m$ productos punto:

```
for i in range(m)
    y[i]+=A[i,:]*x[j] #producto punto
    
```

donde: $A[i,:]$ es el $i$-ésimo renglón de $A$.

* Obsérvese que el acceso a la matriz $A$ es por renglón.

También puede escribirse al algoritmo *gaxpy* en una forma orientada por columnas:

### Algoritmo gaxpy *column oriented*

Este algoritmo ayuda a visualizar al producto matriz-vector como una combinación lineal de las columnas de $A$:

$$Ax = \displaystyle \sum_{j=1}^n a_jx_j$$

con $a_j$ la $j$-ésima columna de $A$.


In [16]:
x=[2]*n
y=[0]*m
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

In [17]:
y

[12.0, 10]

**Obs:**

* El algoritmo de multiplicación matriz-vector (versión *row* o *column* oriented) involucra $\mathcal{O}(mn)$ operaciones o una cantidad **cuadrática** de trabajo, que podemos entender como "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $4$". Tal número de operaciones trabajan sobre una matriz o sobre una cantidad **cuadrática** de datos. A los algoritmos que realizan una cantidad cuadrática de trabajo sobre una cantidad cuadrática de datos se les cataloga de **nivel BLAS 2**. Ver [level 2](http://www.netlib.org/blas/#_level_2) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

* La versión *column oriented* se puede analizar desde el punto de vista puramente algorítmico como un intercambio entre las líneas con los índices $i$ y $j$ de cada *loop* y un acceso a los datos de la matriz por columna. O bien, se puede analizar desde el álgebra lineal indicando que el vector $y$ está en el **espacio generado** por las columnas de $A$ y cuyas coordenadas están siendo dadas por las entradas del vector $x$:

<img src="https://dl.dropboxusercontent.com/s/6a2b7rjs4a71sni/combinacion_lineal_columnas_A.png?dl=0" heigth="700" width="700">

Una ejemplo de visualización del espacio generado por las columnas de $A$, llamado **rango o imagen** de $A$, $Im(A)$, es el siguiente:

<img src="https://dl.dropboxusercontent.com/s/zkbhzv9a2jiw11b/espacio_generado_columnas_de_A.png?dl=0" heigth="400" width="400">

En este dibujo los vectores $b, r(x) \in \mathbb{R}^3$ no están en $Im(A) \subset \mathbb{R}^3$ pero $Ax$ en azul sí.

* Obsérvese que el **inner loop** de la versión *column oriented* en *gaxpy* es un **saxpy** en la que el escalar está dado por una entrada de $x$. Esto lo podemos realizar de forma explícita definiendo $A[:,j]$ a la $j$-ésima columna de $A$ por lo que $A = [A[:,1] | A[:,2] | \dots | A[:,n]]$, entonces:

```
for j in range(n)
    y+=A[:,j] * x[j]
 
```

sin embargo como hemos visto en Python con su implementación más común CPython, no es posible realizar tal indexado:

In [18]:
x=[2]*n
y=[0]*m
for j in range(n):
    y+=A[:,j]*x[j]

TypeError: list indices must be integers or slices, not tuple

a menos que incorporemos alguna paquetería que permita la **vectorización**. Razones a lo anterior tienen que ver con que el [bytecode](https://en.wikipedia.org/wiki/Bytecode) de Python no está optimizado para optimización. Un ejemplo de un paquete que permite realizar operaciones de forma vectorizada es [numpy](https://numpy.org/):

In [19]:
import numpy as np

In [20]:
x = 2*np.ones(n)
y = np.zeros(m)

In [21]:
x

array([2., 2., 2., 2., 2.])

In [22]:
y

array([0., 0.])

In [23]:
A

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]

In [24]:
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [25]:
A

array([[1.2, 1.2, 1.2, 1.2, 1.2],
       [1. , 1. , 1. , 1. , 1. ]])

In [26]:
for j in range(n):
    y+=A[:,j]*x[j]

In [27]:
y

array([12., 10.])

**Comentarios:**

* La vectorización como se describe en [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb), es una herramienta para escribir programas de alto rendimiento pues incrementa\* el número de instrucciones por ciclo [IPC](https://en.wikipedia.org/wiki/Instructions_per_cycle) para procesadores que soportan el Single Instruction Multiple Data (SIMD) de la taxonomía de Flynn (ver [liga](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy)). Como ejemplo de tales procesadores están los procesadores vectoriales o en arreglo, ver [liga](https://en.wikipedia.org/wiki/Vector_processor). Obsérvese en [liga](https://github.com/numpy/numpy/blob/master/numpy/core/src/umath/simd.inc.src) que `numpy` soporta tales operaciones.

\*El incremento en IPC vía la vectorización se enfoca a que las instrucciones tiendan a completar trabajo **útil** por ciclo pues podría darse la situación en la que se tiene: *a high rate of instructions, but a low rate of actual work completed*.


#### `Perf`

En Linux existe la herramienta [perf](https://github.com/torvalds/linux/tree/master/tools/perf) que nos ayuda a calcular métricas de desempeño de la CPU o de los cores. Para la ejecución de los siguientes ejemplos es **indispensable** correr el contenedor de docker en un sistema ubuntu.

**Ejemplo de cálculo de la norma $2$ al cuadrado de un vector**

In [1]:
%%file norm_square.py
n=10**5
vector=list(range(n))
norm=0
for v in vector:
    norm+=v*v

Overwriting norm_square.py


In [2]:
%%bash
sudo perf stat -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

         111619661      cycles                                                        ( +-  0.34% )
         214414673      instructions              #    1.92  insn per cycle           ( +-  0.49% )
           1144583      cache-references                                              ( +-  0.82% )
             54141      cache-misses              #    4.730 % of all cache refs      ( +-  1.22% )

       0.030852668 seconds time elapsed                                          ( +-  0.93% )



**Comentarios:**

* Con `perf` se repiten las mediciones utilizando la *flag* `-r`. Con la *flag* `-e` se enlistan las métricas a calcular. 

* En el ejemplo anterior además de los ciclos e instrucciones se calculan los *cache references* y los *cache misses*. Ver [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb) para definiciones de *cache references* y *misses*. Esencialmente el sistema de memoria buscó el número que aparece en *cache-references* por datos o instrucciones en el caché y de éstas búsquedas, en *cache-misses* se repora el número (y porcentaje) de búsquedas fallidas que no estuvieron en memoria. 

* La fragmentación, ver [Fragmentation](https://en.wikipedia.org/wiki/Fragmentation_(computing)), incrementa el número de transferencias de memoria hacia la CPU y además imposibilita la vectorización porque el caché no está lleno. El caché puede llenarse sólo al tener bloques contiguos de memoria alojados para los datos (recuérdese que la interconexión o *bus* sólo puede mover *chunks* de memoria contigua, ver [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb)). Python en su implementación común CPython, tiene **data fragmentation** por ejemplo al usar listas. Las listas de Python alojan locaciones donde se pueden encontrar los datos y no los datos en sí. Al utilizar tales estructuras para operaciones matriciales el *runtime* de Python debe realizar *lookups* por índices y al no encontrarse de forma contigua los datos, el *overhead* en la transferencia de datos es mayor lo que causa que se tengan mayor número de *cache-misses* y que los cores tengan que esperar hasta que los datos estén disponibles en el caché.

* Obsérvese que se reporta el IPC al lado de *instructions* con nombre *insn per cycle*. Un alto IPC indica una alta transferencia de instrucciones de la unidad de memoria hacia la unidad de cómputo y un menor IPC indica más **stall cycles**, ver [pipeline stall](https://en.wikipedia.org/wiki/Pipeline_stall). Sin embargo como se mencionó antes, se debe evaluar si *a high rate of instructions* indica un *high rate of actual work completed* (p.ej. un *loop* tiene una alta tasa de IPC pero no siempre se realiza trabajo útil). Ver sección *CPU statistics* en la [liga](http://www.brendangregg.com/perf.html).

También podemos obtener las estadísticas por core. Obsérvese que la salida anterior de `perf` calculó el IPC a partir del número de instrucciones y ciclos ahí reportados. El mismo cálculo se realiza a continuación:

In [7]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2          116250518      cycles                                                      
S0-C0           2          216268284      instructions              #    1.86  insn per cycle         
S0-C0           2            1191415      cache-references                                            
S0-C0           2              55711      cache-misses              #    4.676 % of all cache refs    
S0-C1           2             558967      cycles                                                      
S0-C1           2             229116      instructions              #    0.41  insn per cycle         
S0-C1           2               7781      cache-references                                            
S0-C1           2               2947      cache-misses              #   37.874 % of all cache refs    
S0-C2           2             148451      cycles                                                      
S0-C2          

Si ejecutamos `perf` sólo con la *flag* `-r` resulta:

In [8]:
%%bash
sudo perf stat -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

         30.470164      task-clock (msec)         #    0.994 CPUs utilized            ( +-  1.27% )
                 0      context-switches          #    0.008 K/sec                    ( +- 64.07% )
                 0      cpu-migrations            #    0.000 K/sec                  
              2076      page-faults               #    0.068 M/sec                    ( +-  0.02% )
         112922032      cycles                    #    3.706 GHz                      ( +-  0.90% )
         217061916      instructions              #    1.92  insn per cycle           ( +-  1.19% )
          45749113      branches                  # 1501.440 M/sec                    ( +-  1.38% )
            669982      branch-misses             #    1.46% of all branches          ( +-  0.07% )

       0.030654910 seconds time elapsed                                          ( +-  1.26% )



**Comentarios:**

* La métrica de *task-clock* indica cuántos *clock cycles* tomó nuestra tarea y se reporta en milisegundos. Obsérvese que también se indica cuántos CPU's fueron utilizados, no es exactamente 1 pues el programa no sólo involucra trabajo de CPU sino también de alojamiento de memoria. 

* *context-switches* y *cpu-migrations* indican cuánto se detuvo el programa para realizar:

    * operaciones relacionadas con el kernel del sistema (por ejemplo I/O).
    * ejecución de otras aplicaciones.
    * alojamiento de la ejecución en un core distinto (lo que ocasiona que haya movimiento de datos o instrucciones hacia el caché de otro core).
    
la idea es tener un número pequeño de éstas métricas.

* *page-faults* se relaciona con el alojamiento de memoria a partir del kernel del sistema operativo. Ocasiona que se detenga la ejecución del programa y por tanto deseamos que ésta métrica no sea muy grande. Ver [Page fault](https://en.wikipedia.org/wiki/Page_fault).

* Ver [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb) para el *branching*. Esencialmente interesa que el número de *branch misses* sea pequeño.



Podemos generar la salida anterior por *core*:

In [9]:
%%bash
sudo perf stat -a --per-core -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2          62.490719      cpu-clock (msec)          #    2.007 CPUs utilized          
S0-C0           2                  0      context-switches          #    0.000 K/sec                  
S0-C0           2                  0      cpu-migrations            #    0.000 K/sec                  
S0-C0           2                  0      page-faults               #    0.000 K/sec                  
S0-C0           2             688214      cycles                    #    0.011 GHz                    
S0-C0           2             227486      instructions              #    0.33  insn per cycle         
S0-C0           2              40496      branches                  #    0.648 M/sec                  
S0-C0           2               1078      branch-misses             #    2.66% of all branches        
S0-C1           2          62.492165      cpu-clock (msec)          #    2.007 CPUs utilized          
S0-C1          

Para contrastar las salidas anteriores con un mejor uso del caché se utiliza a continuación el paquete de `numpy`:

In [10]:
%%file norm_square_numpy.py
import numpy as np
n=10**5
vector=np.arange(n)
vector.dot(vector)

Overwriting norm_square_numpy.py


In [11]:
%%bash
sudo perf stat -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

        2517687526      cycles                                                        ( +-  0.03% )
        1288139915      instructions              #    0.51  insn per cycle           ( +-  0.10% )
           7543788      cache-references                                              ( +-  0.82% )
            421937      cache-misses              #    5.593 % of all cache refs      ( +-  2.94% )

       0.142374742 seconds time elapsed                                          ( +-  2.07% )



**Comentario:** 

* Obsérvese que es **más tardado** este programa para el número $n$ que se está utilizando en comparación con el programa anterior sin vectorizar.

* El número de instrucciones es **menor** al reportado anteriormente y parece tener un mayor número $%$ de *cache-misses*. Obsérvese que para este ejemplo es más ilustrativo observar las estadísticas por *core* y realizar conclusiones:

In [12]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2          568617624      cycles                                                      
S0-C0           2          221335646      instructions              #    0.39  insn per cycle         
S0-C0           2              35102      cache-references                                            
S0-C0           2               6680      cache-misses              #   19.030 % of all cache refs    
S0-C1           2          815266454      cycles                                                      
S0-C1           2          621661648      instructions              #    0.76  insn per cycle         
S0-C1           2            7426859      cache-references                                            
S0-C1           2             377233      cache-misses              #    5.079 % of all cache refs    
S0-C2           2          570591927      cycles                                                      
S0-C2          

**Comentarios:** 

* Se observa que la métrica IPC está más uniforme a través de los cores lo que indica un mejor *load balancing*.

* Además, el $\%$ de caché misses es menor que en el caso no vectorizado.

También podemos obtener las salidas anteriores con `numpy`: 

In [13]:
%%bash
sudo perf stat -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

        662.336391      task-clock (msec)         #    4.721 CPUs utilized            ( +-  0.05% )
                30      context-switches          #    0.045 K/sec                    ( +-  2.53% )
                 1      cpu-migrations            #    0.002 K/sec                    ( +- 13.78% )
              4753      page-faults               #    0.007 M/sec                    ( +-  0.05% )
        2520283544      cycles                    #    3.805 GHz                      ( +-  0.03% )
        1287882327      instructions              #    0.51  insn per cycle           ( +-  0.09% )
         256426599      branches                  #  387.155 M/sec                    ( +-  0.09% )
           6512414      branch-misses             #    2.54% of all branches          ( +-  0.08% )

       0.140287777 seconds time elapsed                                          ( +-  0.27% )



In [168]:
%%bash
sudo perf stat -a --per-core -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2         276.833831      cpu-clock (msec)          #    2.027 CPUs utilized          
S0-C0           2                 19      context-switches          #    0.069 K/sec                  
S0-C0           2                  3      cpu-migrations            #    0.011 K/sec                  
S0-C0           2               4684      page-faults               #    0.017 M/sec                  
S0-C0           2          842882123      cycles                    #    3.045 GHz                    
S0-C0           2          610775440      instructions              #    0.72  insn per cycle         
S0-C0           2          129130958      branches                  #  466.457 M/sec                  
S0-C0           2            4135105      branch-misses             #    3.20% of all branches        
S0-C1           2         276.834801      cpu-clock (msec)          #    2.027 CPUs utilized          
S0-C1          

### Regresando al algoritmo *gaxpy column oriented*...

Utilizaremos `perf` en lo que sigue para evaluar las métricas de IPC, *cache-references*, *cache-misses* y medir el tiempo de ejecución. Esto permitirá comparar los tiempos de ejecución del algoritmo *gaxpy* (nivel BLAS 2), con vectorización y sin vectorización:

In [178]:
np.random.seed(2020)
m=10**3
n=10**4
A=np.random.rand(m,n)
file='A.txt'
np.savetxt(file,A)

In [179]:
%%file mult_matrix_vector.py
m=10**3
n=10**4
x=[2]*n
y=[0]*m
A = []
file='A.txt'
with open(file,'r') as f:
    for l in f:
        A.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')])      
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

Overwriting mult_matrix_vector.py


In [180]:
%%file mult_matrix_vector_numpy.py
import numpy as np
m=10**3
n=10**4
x = 2*np.ones(n)
y = np.zeros(m)
file='A.txt'
A = np.loadtxt(file)
for j in np.arange(n):
    y+=A[:,j]*x[j]

Overwriting mult_matrix_vector_numpy.py


In [181]:
%%bash
sudo perf stat -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_vector.py


 Performance counter stats for 'python3 mult_matrix_vector.py' (5 runs):

       24552722120      cycles                                                        ( +-  0.94% )
       61319652222      instructions              #    2.50  insn per cycle           ( +-  0.96% )
          93490934      cache-references                                              ( +-  0.22% )
          10556653      cache-misses              #   11.292 % of all cache refs      ( +-  0.30% )

       6.165763093 seconds time elapsed                                          ( +-  0.92% )



In [182]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_vector.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2        24208561977      cycles                                                      
S0-C0           2        60491957822      instructions              #    2.50  insn per cycle         
S0-C0           2           94269284      cache-references                                            
S0-C0           2           10733030      cache-misses              #   11.386 % of all cache refs    
S0-C1           2          134441105      cycles                                                      
S0-C1           2           29975599      instructions              #    0.22  insn per cycle         
S0-C1           2             480718      cache-references                                            
S0-C1           2             195521      cache-misses              #   40.673 % of all cache refs    
S0-C2           2          107643060      cycles                                                      
S0-C2           

In [183]:
%%bash
sudo perf stat -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'python3 mult_matrix_vector_numpy.py' (5 runs):

       26478937473      cycles                                                        ( +-  0.15% )
       62011075836      instructions              #    2.34  insn per cycle           ( +-  0.18% )
          55019889      cache-references                                              ( +-  0.45% )
           7768868      cache-misses              #   14.120 % of all cache refs      ( +-  0.25% )

       6.151853801 seconds time elapsed                                          ( +-  0.14% )



In [184]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2          598633764      cycles                                                      
S0-C0           2          226274215      instructions              #    0.38  insn per cycle         
S0-C0           2             492948      cache-references                                            
S0-C0           2             262961      cache-misses              #   53.345 % of all cache refs    
S0-C1           2        24977156301      cycles                                                      
S0-C1           2        61689232443      instructions              #    2.47  insn per cycle         
S0-C1           2           55841700      cache-references                                            
S0-C1           2            7860595      cache-misses              #   14.077 % of all cache refs    
S0-C2           2         1743371347      cycles                                                      
S0-C2           

* El algoritmo *gaxpy row oriented* puede escribirse de forma más compacta haciendo uso de la definición de producto punto estándar: $x^Ty$ para dos vectores columna $x$ y $y$. En el caso de una matriz $A$ se tiene:

```
for i=1:m
    y[i]+=A[i,:]^T*x
```

y en Python:

In [76]:
x = 2*np.ones(n)
y = np.zeros(m)
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [81]:
for i in range(m):
    y[i]+=A[i,:].dot(x)

In [82]:
y

array([12., 10.])

en donde se utilizó la función [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) de *numpy*.

* 

Matriz matriz

In [185]:
np.random.seed(2020)
m=10**2
r=10**3

A=np.random.rand(m,r)
fileA='A.txt'
np.savetxt(fileA,A)

In [186]:
np.random.seed(2021)
r=10**3
n=10**2

B=np.random.rand(r,n)
fileB='B.txt'
np.savetxt(fileB,B)

In [187]:
%%file mult_matrix_matrix.py
m=10**2
r=10**3
n=10**2

A = []
B = []
fileA='A.txt'
fileB='B.txt'

with open(fileA,'r') as f:
    for l in f:
        A.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')]) 
        
with open(fileB,'r') as f:
    for l in f:
        B.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')])  

C=[[0]*n for i in range(m)]

for i in range(m):
    for j in range(n):
        for k in range(r):
            C[i][j]+=A[i][k]*B[k][j]


Overwriting mult_matrix_matrix.py


In [188]:
%%file mult_matrix_matrix_numpy.py
import numpy as np
m=10**2
r=10**3
n=10**2

fileA='A.txt'
fileB='B.txt'
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)
C = A@B

Overwriting mult_matrix_matrix_numpy.py


In [189]:
%%bash
sudo perf stat  -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2           13045340      cycles                                                      
S0-C0           2            5583447      instructions              #    0.43  insn per cycle         
S0-C0           2             166338      cache-references                                            
S0-C0           2              40524      cache-misses              #   24.362 % of all cache refs    
S0-C1           2         8287660278      cycles                                                      
S0-C1           2        23642328445      instructions              #    2.85  insn per cycle         
S0-C1           2           39051382      cache-references                                            
S0-C1           2             343543      cache-misses              #    0.880 % of all cache refs    
S0-C2           2           12399889      cycles                                                      
S0-C2           

In [190]:
%%bash
sudo perf stat -a -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix.py


 Performance counter stats for 'system wide' (5 runs):

        9076101225      cycles                                                        ( +-  6.73% )
       24394702890      instructions              #    2.69  insn per cycle           ( +-  1.26% )
          39694995      cache-references                                              ( +-  0.75% )
            506669      cache-misses              #    1.276 % of all cache refs      ( +-  8.82% )

       2.099587208 seconds time elapsed                                          ( +-  1.23% )



In [191]:
%%bash
sudo perf stat -a -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy.py


 Performance counter stats for 'system wide' (5 runs):

        3486970954      cycles                                                        ( +-  2.37% )
        2716423165      instructions              #    0.78  insn per cycle           ( +-  0.68% )
           9292320      cache-references                                              ( +-  0.59% )
            596088      cache-misses              #    6.415 % of all cache refs      ( +-  4.75% )

       0.266350340 seconds time elapsed                                          ( +-  2.80% )



In [1]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2         1401257436      cycles                                                      
S0-C0           2         1862709120      instructions              #    1.33  insn per cycle         
S0-C0           2            9134952      cache-references                                            
S0-C0           2             484632      cache-misses              #    5.305 % of all cache refs    
S0-C1           2          706797830      cycles                                                      
S0-C1           2          269257117      instructions              #    0.38  insn per cycle         
S0-C1           2              55035      cache-references                                            
S0-C1           2              21068      cache-misses              #   38.281 % of all cache refs    
S0-C2           2          705777475      cycles                                                      
S0-C2           

In [193]:
%%file mult_matrix_matrix_numpy_dot_product.py
import numpy as np
m=10**2
r=10**3
n=10**2

fileA='A.txt'
fileB='B.txt'
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)
C = np.zeros((m,n))
for i in np.arange(m):
        for j in np.arange(n):
                C[i][j]+= A[i,:].dot(B[:,j])

Overwriting mult_matrix_matrix_numpy_dot_product.py


In [194]:
%%bash
sudo perf stat -a -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (5 runs):

        3052736247      cycles                                                        ( +-  2.05% )
        2680158414      instructions              #    0.88  insn per cycle           ( +-  0.72% )
          11041581      cache-references                                              ( +-  0.80% )
            644429      cache-misses              #    5.836 % of all cache refs      ( +-  7.31% )

       0.282211430 seconds time elapsed                                          ( +-  1.92% )



In [2]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2          569610445      cycles                                                      
S0-C0           2          221505417      instructions              #    0.39  insn per cycle         
S0-C0           2              56811      cache-references                                            
S0-C0           2              17900      cache-misses              #   31.508 % of all cache refs    
S0-C1           2         1319889207      cycles                                                      
S0-C1           2         2002371463      instructions              #    1.52  insn per cycle         
S0-C1           2           10456781      cache-references                                            
S0-C1           2             489184      cache-misses              #    4.678 % of all cache refs    
S0-C2           2          570228997      cycles                                                      
S0-C2           

In [3]:
import os

In [4]:
os.remove('A.txt')
os.remove('B.txt')

**Referencias:**

* E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

* G. H. Golub, C. F. Van Loan, Matrix Computations, John Hopkins University
Press, 2013.

* [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb)

Otras referencias útiles para [perf](https://github.com/torvalds/linux/tree/master/tools/perf) son:

* [perf-wiki](https://perf.wiki.kernel.org/index.php/Tutorial)

* [perf-tools](https://github.com/brendangregg/perf-tools) para *tools* que se apoyan de `perf`.

